### 1. Setup and Imports

In [ ]:
# Install necessary packages
!pip install -Uqq ddgs

# Import all required libraries
import shutil
from ddgs import DDGS
from fastcore.all import *
from fastai.vision.all import *
from fastdownload import download_url
from itertools import islice

print("--- Step 1: Setup complete ---")

### 2. Download and Prepare Data

In [ ]:
# Define the path for our data
path = Path('bird_or_not')

# Force a clean slate by removing the directory if it exists
if path.exists():
    print("Removing existing directory to ensure a clean start.")
    shutil.rmtree(path)

# Function to search for images
def search_images(term, max_images=30):
    print(f"Searching for '{term}'...")
    with DDGS() as ddgs:
        search_results = ddgs.images(term)
        results = list(islice(search_results, max_images))
        return L(results).itemgot('image')

# Search and download images for both classes
searches = 'bird', 'forest'
path.mkdir()

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True)
    results = search_images(f'{o} photo')
    download_images(dest, urls=results)
    resize_images(dest, max_size=400, dest=dest)

# Verify images and remove failures
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
print(f"Found {len(get_image_files(path))} valid images in total.")
print("--- Step 2: Data preparation complete ---")

### 3. Create DataLoaders

In [ ]:
# Create DataLoaders using the DataBlock API
print("Creating DataLoaders with DataBlock...")
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(192, method='squish')
).dataloaders(path, bs=8, num_workers=0) # Use a batch size appropriate for the small dataset

# Show a sample batch of images
dls.show_batch(max_n=8)
print("--- Step 3: DataLoaders created successfully ---")

### 4. Train the Model

In [ ]:
# Train the vision model
print("Training model...")
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)
print("--- Step 4: Model training complete ---")

### 5. Run Inference

In [ ]:
# Use the trained model to make a prediction on a new image
print("Running inference on a new image...")
bird_search = search_images('bird photo', max_images=1)
if bird_search:
    dest = 'bird.jpg'
    download_url(bird_search[0], dest, show_progress=False)
    
    print("Prediction:")
    display(Image.open(dest).to_thumb(256,256))

    is_bird,_,probs = learn.predict(PILImage.create('bird.jpg'))
    print(f"This is a: {is_bird}.")
    print(f"Probability it's a bird: {probs[0]:.4f}")
print("--- Step 5: Inference complete ---")